In [1]:
!pip install lightning datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [2]:
import gdown
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch.utils as utils
from torch.utils.data import DataLoader, random_split

import torchvision
import torchvision.transforms as T
from torchvision.datasets import FashionMNIST

import lightning as L
from lightning import seed_everything
from lightning.pytorch.callbacks import TQDMProgressBar

from datasets import Dataset, load_dataset

import matplotlib as  mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
gdown.download(id='1qd50QDZtr_NYFiFVdp0sIvGDwTT3mMEQ')
!mkdir ../data
!mv lego-brick-images.zip ../data
!unzip -q ../data/lego-brick-images.zip

Downloading...
From: https://drive.google.com/uc?id=1qd50QDZtr_NYFiFVdp0sIvGDwTT3mMEQ
To: /content/lego-brick-images.zip
100%|██████████| 1.07G/1.07G [00:09<00:00, 118MB/s]


In [4]:
seed_everything(42)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

In [15]:
class DCGAN(L.LightningModule):
    def __init__(self, latent_dim=100, gradient_weight=10.0):
        super().__init__()
        self.automatic_optimization = False
        self.latent_dim = latent_dim
        self.gradient_weight = gradient_weight
        self.generator = nn.Sequential(
            nn.Unflatten(1, (self.latent_dim, 1, 1)),
            nn.ConvTranspose2d(self.latent_dim, 512, (4, 4), stride=1),
            nn.BatchNorm2d(512, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(512, 256, (4, 4), stride=2, padding=(1, 1)),
            nn.BatchNorm2d(256, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(256, 128, (4, 4), stride=2, padding=(1, 1)),
            nn.BatchNorm2d(128, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(128, 64, (4, 4), stride=2, padding=(1, 1), bias=False),
            nn.BatchNorm2d(64, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64, 1, (4, 4), stride=2, padding=(1, 1), bias=False),
            nn.Tanh()
        )
        self.discriminator = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(4, 4), stride=2, padding=(2, 2), bias=False),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(0.3),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=2, padding=(2, 2), bias=False),
            nn.BatchNorm2d(128, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(0.3),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=2, padding=(2, 2), bias=False),
            nn.BatchNorm2d(256, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(0.3),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=2, padding=(2, 2), bias=False),
            nn.BatchNorm2d(512, momentum=0.9),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(0.3),
            nn.Conv2d(512, 1, kernel_size=(4, 4), bias=False),
            nn.Flatten(),
            nn.Linear(4, 1),
        )

    def forward(self, x):
        img_tensor = self.generator(x)
        return img_tensor

    def configure_optimizers(self):
        gen_opt = Adam(self.generator.parameters(), lr=2e-4)
        dsc_opt = Adam(self.discriminator.parameters(), lr=2e-4)
        return [gen_opt, dsc_opt], []

    def generative_loss(self, pred):
        return torch.log(pred).mean()

    def discriminative_loss(self, real_pred, fake_pred):
        return - torch.log(1 - fake_pred).mean() - torch.log(real_pred).mean()

    def gradient_penalty(self, real, fake):
        alpha = torch.randn([real.shape[0], 1, 1, 1]).type_as(real)
        diff = fake - real
        interpolated = real + alpha * diff
        interpolated.requires_grad = True
        pred = self.discriminator(interpolated)
        gradient = torch.autograd.grad(
            outputs=pred,
            inputs=interpolated,
            grad_outputs=torch.ones_like(pred),
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]
        gradient = gradient.view(gradient.shape[0], -1)
        gradient_norm = torch.sqrt(torch.sum(torch.pow(gradient, 2), dim=1))
        penalty = torch.mean((gradient_norm - 1) ** 2)
        return penalty

    def training_step(self, batch, batch_idx):
        X = batch['image']
        gen_opt, dsc_opt = self.optimizers()

        self.toggle_optimizer(dsc_opt)

        for i in range(3):
            z = torch.randn(X.shape[0], self.latent_dim).type_as(X)
            fake_img = self.generator(z)
            real_pred = self.discriminator(X)
            fake_pred = self.discriminator(fake_img)
            wasserstein_loss = self.discriminative_loss(real_pred, fake_pred)
            gradient_penalty = self.gradient_penalty(X, fake_img)
            dsc_loss = wasserstein_loss + self.gradient_weight * gradient_penalty

            self.manual_backward(dsc_loss)
            dsc_opt.step()
            dsc_opt.zero_grad()

        self.untoggle_optimizer(dsc_opt)

        self.toggle_optimizer(gen_opt)

        z = torch.randn(X.shape[0], self.latent_dim).type_as(X)
        fake_img = self.generator(z)
        fake_pred = self.discriminator(fake_img)
        gen_loss = self.generative_loss(fake_pred)

        self.manual_backward(gen_loss)
        gen_opt.step()
        gen_opt.zero_grad()
        self.untoggle_optimizer(gen_opt)

In [6]:
transform = T.Compose([
    T.Resize((64, 64)),
    T.Grayscale(num_output_channels=1),
    T.ToTensor(),
    T.Normalize((0.5,), (0.5,))
])

def to_tensor(row):
    return {
        'image': transform(row['image'])
    }

In [7]:
train_dataset = load_dataset('./dataset')['train'].map(to_tensor).with_format('torch')

Resolving data files:   0%|          | 0/40000 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=128 * 2, shuffle=True, num_workers=8)

In [16]:
trainer = L.Trainer(max_epochs=30, callbacks=[TQDMProgressBar(refresh_rate=1)], accelerator='auto')
model = DCGAN()

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [17]:
trainer.fit(model, train_dataloaders=train_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 3.6 M 
1 | discriminator | Sequential | 2.8 M 
---------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.356    Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 3.6 M 
1 | discriminator | Sequential | 2.8 M 
---------------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.356    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=30` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.
